## Final submission notebook for  exercise 3

#### Student name: Karan Sanjay Dhage
#### student Id: 419197

# Data prepartion for SIR model

In [ ]:

from datetime import datetime

import numpy as np
import pandas as pd


def Sir_model_data():
    data_path = r'C:\Users\HP\Desktop\pythonProject/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series' \
                r'/time_series_covid19_confirmed_global.csv '
    pd_raw = pd.read_csv(data_path)
    time_idx = pd_raw.columns[4:]
    df_plot = pd.DataFrame({
        'date': time_idx})
    country_list = ['India',
                    'US',
                    'Spain',
                    'Germany',
                    'France',
                    ]
    for each in country_list:
        df_plot[each] = np.array(pd_raw[pd_raw['Country/Region'] == each].iloc[:, 4::].sum(axis=0))
    time_idx = [datetime.strptime(each, "%m/%d/%y") for each in df_plot.date]  # convert to datetime
    time_str = [each.strftime('%Y-%m-%d') for each in time_idx]  # convert back to date ISO norm (str)
    df_plot['date'] = time_idx
    #print(df_plot)
    df_plot.to_csv(r'C:\Users\HP\Desktop\pythonProject/data/processed/COVID_small_flat_table.csv', sep=';', index=False)


if __name__ == '__main__':
    Sir_model_data()

# Developement of SIR model and Dashboard

In [ ]:

import numpy as np
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
from scipy import optimize
from scipy import integrate
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)


def SIR_model_t(SIR, t, beta, gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta:

        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)

    '''
    global N0
    S, I, R = SIR
    dS_dt = -beta * S * I / N0  # S*I is the
    dI_dt = beta * S * I / N0 - gamma * I
    dR_dt = gamma * I
    return dS_dt, dI_dt, dR_dt


def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), x, args=(beta, gamma))[:, 1]  # we only would like to get dI


df_analyse = pd.read_csv(r'C:\Users\HP\Desktop\pythonProject/data/processed/COVID_small_flat_table.csv', sep=';')
df_analyse.sort_values('date', ascending=True)

dash.__version__

fig = go.Figure()
app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
        # SIR model for Applied Data Science on COVID-19 dataset
        SIR model was developed for COVID-19 dataset and implemented in this dashboard. 

        '''),
    dcc.Markdown('''
        ## Selection of various countries for visualization purpose
        '''),

    dcc.Dropdown(
        id='countries',
        options=[{'label': each, 'value': each} for each in df_analyse.columns[1:]],
        value=['Germany', 'US', 'Spain'],  # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
            ## Graphical representation of SIR model
            '''),

    dcc.Graph(figure=fig, id='main_window_slope'),
])


@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('countries', 'value')])
def update_figure(country_list):
    traces = []

    my_yaxis = {'type': "log",
                'title': 'Total population'
                }
    for each in country_list:
        Col = df_analyse[each]
        ydata = np.array(Col[40:150])
        global I0, S0, R0, N0
        N0 = 10000000
        t = np.arange(len(ydata))
        I0 = ydata[0]
        S0 = N0 - I0
        R0 = 0
        ## get the model parameters / fit the model
        popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
        # get the final fitted curve / predict the outcome
        fitted = fit_odeint(t, *popt)
        traces.append(dict(x=t,
                           y=np.ediff1d(ydata, to_begin=ydata[1] - ydata[0]),
                           mode='markers',
                           opacity=0.9,
                           name=each
                           )
                      )
        traces.append(dict(x=t,
                           y=fitted,
                           mode='lines',
                           opacity=0.9,
                           name=each
                           )
                      )

    return {
        'data': traces,
        'layout': dict(
            width=1280,
            height=720,
            title="SIR Simulation fit curve for selected countries",
            xaxis={'title': 'Timeline',
                   'tickangle': -45,
                   'nticks': 20,
                   'tickfont': dict(size=14, color="#7f7f7f"),
                   },

            yaxis=my_yaxis
        )
    }


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\Users\HP\anaconda3\lib\site-packages\scipy\integrate\odepack.py:248: ODEintWarning:

Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\HP\anaconda3\lib\site-packages\scipy\integrate\odepack.py:248: ODEintWarning:

Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.

